In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from scipy.stats import beta
from snowflake import connector
import pandas as pd
import os
from dotenv import load_dotenv

from tabulate import tabulate

import openai

In [2]:
load_dotenv('/Users/peter/.env')
username = os.getenv("SNOWFLAKE_USERNAME")
password=os.getenv("SNOWFLAKE_PASSWORD")
account=os.getenv("SNOWFLAKE_ACCOUNT")
role=os.getenv("SNOWFLAKE_ROLE"), 
warehouse=os.getenv("SNOWFLAKE_WAREHOUSE")

# establish Snowflake connection
connection = connector.connect(user=os.getenv("SNOWFLAKE_USERNAME"), 
                  password=os.getenv("SNOWFLAKE_PASSWORD"), 
                  account=os.getenv("SNOWFLAKE_ACCOUNT"), 
                  role=os.getenv("SNOWFLAKE_ROLE"), 
                  warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
                  database = 'DISCO_CORE' 
                  )

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

## Setup SQL Queries

### Daily Spend & Conversion Orders

In [3]:
daily_spend_query = ''' 
WITH 
bod_individual_conversions_overdelivery AS (
    select 
        bodod.*,
        mc.driving_brand,
        mc.order_cost as order_revenue,
        mc.customer_type,
        mc.time_delta_days,
        mc.most_recent_purchase_publisher
    from disco_mixpanel.bod_individual_conversions_overdelivery as bodod
    left join disco_mixpanel.master_conversions mc on bodod.conversion_unique_key = mc.unique_key
)
, merchant_ids as (
    select 
        cb.name as merchant_name,
        cb.core_brand_id as brand_id,
        cp.remote_id as publisher_remote_id,
        cp.core_publisher_id as publisher_id,
        cp.widget_live_date as widget_live_date,
        min(ldp.date) as first_dfl_date
    from postgres.core_brand as cb
        inner join postgres.core_publisher as cp on cp.brand_id = cb.core_brand_id
        left join disco_mixpanel.loads_displays_performance as ldp on ldp.merchant = cb.name and widget_loads_d > 0
    group by 1,2,3,4,5
)
select 
    (TO_CHAR(TO_DATE(bico.conversion_order_time ), 'YYYY-MM-DD')) AS date,
    COALESCE (SUM(bico.bod_spend ), 0) AS total_bod_spend
from bod_individual_conversions_overdelivery bico 
left join merchant_ids mi on bico.brand_id = mi.brand_id
WHERE 
    ((( bico.conversion_order_time ) >= ((DATEADD('day', -59, CURRENT_DATE()))) 
        AND ( bico.conversion_order_time ) < ((DATEADD('day', 60, DATEADD('day', -59, CURRENT_DATE())))))) 
    AND ((TO_DATE(bico.conversion_order_time )) < TO_DATE(CURRENT_TIMESTAMP())) 
    AND (mi.merchant_name) IS NOT NULL
GROUP BY
    (TO_DATE(bico.conversion_order_time ))
ORDER BY 1 DESC
limit 50;
'''

daily_spend = pd.read_sql(daily_spend_query, connection)
daily_spend.head()

/var/folders/nn/ckstnx1110j6y5fknhgmvlvr0000gn/T/ipykernel_84305/1456628682.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  daily_spend = pd.read_sql(daily_spend_query, connection)


,DATE,TOTAL_BOD_SPEND
0,2024-03-26,4192.38
1,2024-03-25,4798.30
2,2024-03-24,5208.49
3,2024-03-23,4821.16
4,2024-03-22,3809.34


In [23]:
daily_orders_query = '''
SELECT
    (TO_CHAR(TO_DATE(order_data_flattened."CREATED_AT_GMT"), 'YYYY-MM-DD')) AS date,
    COUNT(DISTINCT ( order_data_flattened."ID" ) ) AS unique_orders,
    COUNT(*) AS "order_data_flattened.count"
FROM "ORDERS"."ORDER_DATA_FLATTENED"
AS order_data_flattened
WHERE ((( order_data_flattened."CREATED_AT" ) >= ((DATEADD('day', -59, CURRENT_DATE()))) AND ( order_data_flattened."CREATED_AT" ) < ((DATEADD('day', 60, DATEADD('day', -59, CURRENT_DATE())))))) AND ((( order_data_flattened
."CREATED_AT_GMT" ) >= ((DATEADD('day', -59, CURRENT_DATE()))) AND ( order_data_flattened."CREATED_AT_GMT" ) < ((DATEADD('day', 60, DATEADD('day', -59, CURRENT_DATE())))))) AND (order_data_flattened."ORDER_COST" ) < 100000 AND
((( order_data_flattened."ORDER_SOURCE_NAME" ) NOT LIKE '%shopify_draft_order%' ESCAPE '*' OR ( order_data_flattened."ORDER_SOURCE_NAME" ) IS NULL)) AND ((TO_DATE(order_data_flattened."CREATED_AT_GMT")) < TO_DATE
(CURRENT_TIMESTAMP()))
GROUP BY
    (TO_DATE(order_data_flattened."CREATED_AT_GMT"))
ORDER BY
    1 DESC
limit 50;
''' 
daily_orders = pd.read_sql(daily_orders_query, connection)
daily_orders.head()

/var/folders/nn/ckstnx1110j6y5fknhgmvlvr0000gn/T/ipykernel_6779/3597093423.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  daily_orders = pd.read_sql(daily_orders_query, connection)


,DATE,UNIQUE_ORDERS,order_data_flattened.count
0,2024-03-04,154549,154549
1,2024-03-03,156923,156923
2,2024-03-02,164791,164791
3,2024-03-01,190443,190443
4,2024-02-29,204489,204489


In [27]:
daily_order_spend = daily_spend.merge(daily_orders, on = 'DATE', how = 'left')
daily_order_spend.head(10)

,DATE,TOTAL_BOD_SPEND,UNIQUE_ORDERS,order_data_flattened.count
0,2024-03-04,4444.65,154549,154549
1,2024-03-03,6035.28,156923,156923
2,2024-03-02,6814.56,164791,164791
3,2024-03-01,6322.86,190443,190443
4,2024-02-29,8427.56,204489,204489
5,2024-02-28,6857.78,169434,169434
6,2024-02-27,6755.15,155865,155865
7,2024-02-26,8223.74,152912,152912
8,2024-02-25,6833.20,154670,154670
9,2024-02-24,6856.56,152718,152718


In [29]:
#Calculate the mean % change in spend as an absolute value for the last 50 days
daily_order_spend['TOTAL_BOD_SPEND'].mean()

6997.2006

In [46]:
def calculate_change_metrics(df, metric):
    if metric == 'spend':
        metric_column = 'TOTAL_BOD_SPEND'
        change_column = 'pct_change_spend'
    elif metric == 'orders':
        metric_column = 'UNIQUE_ORDERS'
        change_column = 'pct_change_orders'

    # Ensure the date column is in datetime format
    df['DATE'] = pd.to_datetime(df['DATE'])
    
    # Sort the DataFrame by date to ensure correct calculation of percentage change
    df = df.sort_values('DATE')

    # Calculate % change in daily spend
    df[change_column] = round(df[metric_column].pct_change(),4) ####* 100
    
    # Calculate the standard deviation of the % change
    std_dev_pct_change = df[change_column].std()

    # Calculate absolute difference between daily % change and standard deviation
    df['change_outside_range'] = round((abs(df[change_column]) - std_dev_pct_change),4)

    #Change all the values less than 0 to 0
    df['change_outside_range'] = df['change_outside_range'].apply(lambda x: 0 if x < 0 else x)

    #If the absolute difference is greater than the standard deviation, then flag in a new column as True
    #df['significant_change_flag'] = df['change_outside_range'] > 0

    # Standard deviation as a fixed value across all rows
    df['std_dev_pos'] = std_dev_pct_change
    df['std_dev_neg'] = std_dev_pct_change * -1

    # Return the relevant columns for visualization
    return df.sort_values('DATE', ascending = False)

daily_spend_change = calculate_change_metrics(daily_spend, metric='spend')
daily_spend_change.head()

,DATE,TOTAL_BOD_SPEND,pct_change_spend,change_outside_range,std_dev_pos,std_dev_neg
0,2024-03-04,4444.65,-0.2636,0.1227,0.140879,-0.140879
1,2024-03-03,6035.28,-0.1144,0.0000,0.140879,-0.140879
2,2024-03-02,6814.56,0.0778,0.0000,0.140879,-0.140879
3,2024-03-01,6322.86,-0.2497,0.1088,0.140879,-0.140879
4,2024-02-29,8427.56,0.2289,0.0880,0.140879,-0.140879


In [72]:
daily_spend_change.head()

,DATE,TOTAL_BOD_SPEND,pct_change_spend,change_outside_range,std_dev_pos,std_dev_neg
0,2024-03-04,4444.65,-0.2636,0.1227,0.140879,-0.140879
1,2024-03-03,6035.28,-0.1144,0.0000,0.140879,-0.140879
2,2024-03-02,6814.56,0.0778,0.0000,0.140879,-0.140879
3,2024-03-01,6322.86,-0.2497,0.1088,0.140879,-0.140879
4,2024-02-29,8427.56,0.2289,0.0880,0.140879,-0.140879


In [48]:
daily_order_change = calculate_change_metrics(daily_orders, metric='orders')
daily_order_change.head()

,DATE,UNIQUE_ORDERS,order_data_flattened.count,pct_change_orders,change_outside_range,std_dev_pos,std_dev_neg
0,2024-03-04,154549,154549,-0.0151,0.0000,0.081267,-0.081267
1,2024-03-03,156923,156923,-0.0477,0.0000,0.081267,-0.081267
2,2024-03-02,164791,164791,-0.1347,0.0534,0.081267,-0.081267
3,2024-03-01,190443,190443,-0.0687,0.0000,0.081267,-0.081267
4,2024-02-29,204489,204489,0.2069,0.1256,0.081267,-0.081267


### Publisher Data

In [61]:
publisher_query = ''' 
WITH merchant_ids AS (select cb.name as merchant_name,
                cb.core_brand_id as brand_id,
                cp.remote_id as publisher_remote_id,
                cp.core_publisher_id as publisher_id,
                cp.widget_live_date as widget_live_date,
                min(ldp.date) as first_dfl_date
      from postgres.core_brand as cb
        inner join postgres.core_publisher as cp on cp.brand_id = cb.core_brand_id
        left join disco_mixpanel.loads_displays_performance as ldp on ldp.merchant = cb.name and widget_loads_d > 0
      group by 1, 2, 3, 4, 5
            )
SELECT
    case when loads_displays_performance.merchant = 'Péla Case' then 'Pela Case'
              when loads_displays_performance.merchant = '120/Life Beverage' then '120/Life'
              when loads_displays_performance.merchant = 'BAZZAAL BOX' then 'BAZZAAL INC'
              when loads_displays_performance.merchant = 'California Arts & Recreation Board' then 'California Arts'
              when loads_displays_performance.merchant = 'CanDo (makers of Keto Krisp)' then 'CanDo'
              when loads_displays_performance.merchant = 'CanDo (makes of Keto Krisp)' then 'CanDo'
              when loads_displays_performance.merchant = 'Irecovr' then 'iRecovr'
              when loads_displays_performance.merchant = 'Laura Geller Beauty' then 'Laura Geller'
              when loads_displays_performance.merchant = 'Pet Memory Shop' then 'Pet Memory'
              when loads_displays_performance.merchant = 'The Crunch Cup' then 'The CrunchCup'
              when loads_displays_performance.merchant = 'Tippsy, Inc' then 'Tippsy Sake'
              when loads_displays_performance.merchant = 'WHY AND WHALE' then 'Why and Whale'
              else loads_displays_performance.merchant
              end  AS "loads_displays_performance.merchant",
    companies."PROPERTY_BRAND_SEGMENTATION":value::string  AS "companies.property_brand_segmentation",
    CASE WHEN (companies."PROPERTY_BRAND_SEGMENTATION":value::string) = 'High' THEN 1 ELSE CASE WHEN (companies."PROPERTY_BRAND_SEGMENTATION":value::string) = 'Medium' THEN 2 ELSE CASE WHEN (companies."PROPERTY_BRAND_SEGMENTATION":value::string) = 'Low' THEN 3 ELSE CASE WHEN (companies."PROPERTY_BRAND_SEGMENTATION":value::string) = 'TBS' THEN 4 ELSE 5 END
 END
 END
 END
 AS brand_segmentation_ordered,
        (TO_CHAR(TO_DATE(loads_displays_performance.date ), 'YYYY-MM-DD')) AS "loads_displays_performance.date_date",
            COALESCE(SUM(loads_displays_performance.cross_sell_email_brand_displays_d ), 0) + COALESCE(SUM(loads_displays_performance.cross_sell_widget_brand_displays_d ), 0) + COALESCE(SUM(loads_displays_performance.collective_brand_displays_d ), 0) AS "loads_displays_performance.total_cross_sell_brand_displays"
FROM disco_mixpanel.loads_displays_performance  AS loads_displays_performance
LEFT JOIN merchant_ids ON (case when loads_displays_performance.merchant = 'Péla Case' then 'Pela Case'
              when loads_displays_performance.merchant = '120/Life Beverage' then '120/Life'
              when loads_displays_performance.merchant = 'BAZZAAL BOX' then 'BAZZAAL INC'
              when loads_displays_performance.merchant = 'California Arts & Recreation Board' then 'California Arts'
              when loads_displays_performance.merchant = 'CanDo (makers of Keto Krisp)' then 'CanDo'
              when loads_displays_performance.merchant = 'CanDo (makes of Keto Krisp)' then 'CanDo'
              when loads_displays_performance.merchant = 'Irecovr' then 'iRecovr'
              when loads_displays_performance.merchant = 'Laura Geller Beauty' then 'Laura Geller'
              when loads_displays_performance.merchant = 'Pet Memory Shop' then 'Pet Memory'
              when loads_displays_performance.merchant = 'The Crunch Cup' then 'The CrunchCup'
              when loads_displays_performance.merchant = 'Tippsy, Inc' then 'Tippsy Sake'
              when loads_displays_performance.merchant = 'WHY AND WHALE' then 'Why and Whale'
              else loads_displays_performance.merchant
              end) = merchant_ids.merchant_name
LEFT JOIN "HUBSPOT_DATA"."COMPANIES"  AS companies ON merchant_ids.publisher_remote_id = (companies."PROPERTY_DISCO_ID":value::string)
WHERE ((( loads_displays_performance.date  ) >= ((TO_DATE(DATEADD('day', -4, CURRENT_DATE())))) AND ( loads_displays_performance.date  ) < ((TO_DATE(DATEADD('day', 5, DATEADD('day', -4, CURRENT_DATE()))))))) AND (case when loads_displays_performance.merchant = 'Péla Case' then 'Pela Case'
              when loads_displays_performance.merchant = '120/Life Beverage' then '120/Life'
              when loads_displays_performance.merchant = 'BAZZAAL BOX' then 'BAZZAAL INC'
              when loads_displays_performance.merchant = 'California Arts & Recreation Board' then 'California Arts'
              when loads_displays_performance.merchant = 'CanDo (makers of Keto Krisp)' then 'CanDo'
              when loads_displays_performance.merchant = 'CanDo (makes of Keto Krisp)' then 'CanDo'
              when loads_displays_performance.merchant = 'Irecovr' then 'iRecovr'
              when loads_displays_performance.merchant = 'Laura Geller Beauty' then 'Laura Geller'
              when loads_displays_performance.merchant = 'Pet Memory Shop' then 'Pet Memory'
              when loads_displays_performance.merchant = 'The Crunch Cup' then 'The CrunchCup'
              when loads_displays_performance.merchant = 'Tippsy, Inc' then 'Tippsy Sake'
              when loads_displays_performance.merchant = 'WHY AND WHALE' then 'Why and Whale'
              else loads_displays_performance.merchant
              end ) IS NOT NULL
GROUP BY
    (TO_DATE(loads_displays_performance.date )),1,2,3
ORDER BY
    3,1,4
limit 100;
'''
publisher_data = pd.read_sql(publisher_query, connection)
publisher_data

/var/folders/nn/ckstnx1110j6y5fknhgmvlvr0000gn/T/ipykernel_6779/4045812489.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  publisher_data = pd.read_sql(publisher_query, connection)


,loads_displays_performance.merchant,companies.property_brand_segmentation,BRAND_SEGMENTATION_ORDERED,loads_displays_performance.date_date,loads_displays_performance.total_cross_sell_brand_displays
0,Amberjack,High,1,2024-03-01,1346
1,Amberjack,High,1,2024-03-02,1048
2,Amberjack,High,1,2024-03-03,1049
3,Amberjack,High,1,2024-03-04,1387
4,Andie,High,1,2024-03-01,0
...,...,...,...,...,...
95,HATCH,High,1,2024-03-04,533
96,INH Hair,High,1,2024-03-01,244
97,INH Hair,High,1,2024-03-02,532
98,INH Hair,High,1,2024-03-03,327


In [63]:
#Find the top 5 merchants by total displays
publisher_data.groupby('loads_displays_performance.merchant')['loads_displays_performance.total_cross_sell_brand_displays'].sum().sort_values(ascending = False).head(5)

loads_displays_performance.merchant
BlendJet       50606
Dr. Squatch    42947
Glamnetic      20521
Bodily          8249
Blueland        7705
Name: loads_displays_performance.total_cross_sell_brand_displays, dtype: int64

In [82]:
#identify the top 5 merchants by total displays
top_merchants = publisher_data.groupby('loads_displays_performance.merchant')['loads_displays_performance.total_cross_sell_brand_displays'].sum().sort_values(ascending = False).head(5).index
#if the row's loads_displays_performance.merchant is not in the top_merchants list, then change the loads_displays_performance.merchant to 'Other'
publisher_data['loads_displays_performance.merchant'] = publisher_data['loads_displays_performance.merchant'].apply(lambda x: x if x in top_merchants else 'Other')
#group by date and merchant and sum the total displays
brand_display_df = publisher_data.groupby(['loads_displays_performance.date_date','loads_displays_performance.merchant'])['loads_displays_performance.total_cross_sell_brand_displays'].sum().reset_index()
brand_display_df.columns = ['date', 'top_merchant', 'total_cross_sell_displays']
brand_display_df.head()

,date,top_merchant,total_cross_sell_displays
0,2024-03-01,BlendJet,13169
1,2024-03-01,Bodily,1963
2,2024-03-01,Dr. Squatch,14199
3,2024-03-01,Glamnetic,4834
4,2024-03-01,Other,11104


In [84]:
def calculate_brand_display_metrics(df):
    # Ensure the date column is in datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    # Sort the DataFrame by date and merchant to ensure correct calculation of percentage change
    df = df.sort_values(['top_merchant', 'date'])

    # Calculate % change in displays for each merchant
    df['pct_change_displays'] = round(df.groupby('top_merchant')['total_cross_sell_displays'].pct_change(),4)

    # Calculate the standard deviation of the % change for each merchant
    std_deviation = df.groupby('top_merchant')['pct_change_displays'].std().reset_index(name='std_dev_pct_change')

    # Merge the standard deviation back into the original DataFrame
    df = df.merge(std_deviation, on='top_merchant', how='left')

    # Calculate the difference between the daily % change and standard deviation
    df['change_outside_range'] = round(abs(df['pct_change_displays']) - df['std_dev_pct_change'],4)
    #Change all the values less than 0 to 0
    df['change_outside_range'] = df['change_outside_range'].apply(lambda x: 0 if x < 0 else x)

    # Return the DataFrame with the necessary columns
    return df

brand_df = calculate_brand_display_metrics(brand_display_df)
brand_df

,date,top_merchant,total_cross_sell_displays,pct_change_displays,std_dev_pct_change,change_outside_range
0,2024-03-01,BlendJet,13169,NaN,0.224638,NaN
1,2024-03-02,BlendJet,12993,-0.0134,0.224638,0.0000
2,2024-03-03,BlendJet,10761,-0.1718,0.224638,0.0000
3,2024-03-04,BlendJet,13683,0.2715,0.224638,0.0469
4,2024-03-01,Bodily,1963,NaN,0.191368,NaN
5,2024-03-02,Bodily,1888,-0.0382,0.191368,0.0000
6,2024-03-03,Bodily,1901,0.0069,0.191368,0.0000
7,2024-03-04,Bodily,2497,0.3135,0.191368,0.1221
8,2024-03-01,Dr. Squatch,14199,NaN,0.388097,NaN
9,2024-03-02,Dr. Squatch,9261,-0.3478,0.388097,0.0000


## Language Model summary

In [81]:
spend_output = daily_spend_change.drop(columns =['std_dev_neg'])
spend_output.columns = ['Date', 'DailyRevenue', 'Daily_%_Change', '%_Change_Outside_StdDev', 'Std_Dev']
print(tabulate(spend_output.head(10), headers='keys', tablefmt='plain', showindex=False))

Date                   DailyRevenue    Daily_%_Change    %_Change_Outside_StdDev    Std_Dev
2024-03-04 00:00:00         4444.65           -0.2636                     0.1227   0.140879
2024-03-03 00:00:00         6035.28           -0.1144                     0        0.140879
2024-03-02 00:00:00         6814.56            0.0778                     0        0.140879
2024-03-01 00:00:00         6322.86           -0.2497                     0.1088   0.140879
2024-02-29 00:00:00         8427.56            0.2289                     0.088    0.140879
2024-02-28 00:00:00         6857.78            0.0152                     0        0.140879
2024-02-27 00:00:00         6755.15           -0.1786                     0.0377   0.140879
2024-02-26 00:00:00         8223.74            0.2035                     0.0626   0.140879
2024-02-25 00:00:00         6833.2            -0.0034                     0        0.140879
2024-02-24 00:00:00         6856.56            0.1577                     0.0168

In [79]:
order_output = daily_order_change[['DATE', 'UNIQUE_ORDERS', 'pct_change_orders', 'std_dev_pos', 'change_outside_range']]
order_output.columns = ['Date', 'Order_Count', 'Daily_%_Change', 'StdDev', '%_Change_Outside_StdDev']

print(tabulate(order_output.head(10), headers='keys', tablefmt='plain', showindex=False))

Date                   Order_Count    Daily_%_Change     StdDev    %_Change_Outside_StdDev
2024-03-04 00:00:00         154549           -0.0151  0.0812668                     0
2024-03-03 00:00:00         156923           -0.0477  0.0812668                     0
2024-03-02 00:00:00         164791           -0.1347  0.0812668                     0.0534
2024-03-01 00:00:00         190443           -0.0687  0.0812668                     0
2024-02-29 00:00:00         204489            0.2069  0.0812668                     0.1256
2024-02-28 00:00:00         169434            0.0871  0.0812668                     0.0058
2024-02-27 00:00:00         155865            0.0193  0.0812668                     0
2024-02-26 00:00:00         152912           -0.0114  0.0812668                     0
2024-02-25 00:00:00         154670            0.0128  0.0812668                     0
2024-02-24 00:00:00         152718           -0.0221  0.0812668                     0


In [86]:

print(tabulate(brand_df, headers='keys', tablefmt='plain', showindex=False))


date                 top_merchant      total_cross_sell_displays    pct_change_displays    std_dev_pct_change    change_outside_range
2024-03-01 00:00:00  BlendJet                              13169               nan                   0.224638                nan
2024-03-02 00:00:00  BlendJet                              12993                -0.0134              0.224638                  0
2024-03-03 00:00:00  BlendJet                              10761                -0.1718              0.224638                  0
2024-03-04 00:00:00  BlendJet                              13683                 0.2715              0.224638                  0.0469
2024-03-01 00:00:00  Bodily                                 1963               nan                   0.191368                nan
2024-03-02 00:00:00  Bodily                                 1888                -0.0382              0.191368                  0
2024-03-03 00:00:00  Bodily                                 1901                 0.0069

In [ ]:
def get_completion(instruction_prompt, input_prompt, model = 'gpt-3.5-turbo'):
        response = openai.ChatCompletion.create(
            model= model,
            messages=[
                {
                "role": "system",
                "content": instruction_prompt
                },
                {
                "role": "user",
                "content": input_prompt
                }
            ],
            temperature=1,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
            )
        
        return response['choices'][0]['message']['content']

#### Simple Example

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "system",
      "content": "You are an AI assistant for an online advertising startup tasked with providing a daily summary of the company's key performance metrics, and to provide context and details on why performance improved or worsened. \n\nFor context, the company's business model is a recommended product feed which are displayed on a purchase confirmation page. The product feed includes multiple brands, and the company makes money when a user makes purchase from one of these recommendations. \n\nAt the high-level, the most important health metric for the company is total revenue. Next, the other important metric is the total number of orders (made via recommendations). I will be providing you both of these metrics at the daily level along with other metrics like the daily % change, standard deviation of daily % change, and if the daily % change is within this standard deviation (and if not, what the % difference between the daily change and the standard deviation was). \n\nBased on this information, you should be able to gather on a high-level what the daily performance looks like compared to the look-back window of the data, and call out any major trends.\n\nKeep the tone business professional and insight concise."
    },
    {
      "role": "user",
      "content": "Today's Date: 2024-03-03\n\nRevenue data:\nDate                   DailyRevenue    Daily_%_Change    %_Change_Outside_StdDev    Std_Dev\n2024-03-04 00:00:00         4444.65           -0.2636                     0.1227   0.140879\n2024-03-03 00:00:00         6035.28           -0.1144                     0        0.140879\n2024-03-02 00:00:00         6814.56            0.0778                     0        0.140879\n2024-03-01 00:00:00         6322.86           -0.2497                     0.1088   0.140879\n2024-02-29 00:00:00         8427.56            0.2289                     0.088    0.140879\n2024-02-28 00:00:00         6857.78            0.0152                     0        0.140879\n2024-02-27 00:00:00         6755.15           -0.1786                     0.0377   0.140879\n2024-02-26 00:00:00         8223.74            0.2035                     0.0626   0.140879\n2024-02-25 00:00:00         6833.2            -0.0034                     0        0.140879\n2024-02-24 00:00:00         6856.56            0.1577                     0.0168   0.140879\n\nOrders data: \nDate                   Order_Count    Daily_%_Change     StdDev    %_Change_Outside_StdDev\n2024-03-04 00:00:00         154549           -0.0151  0.0812668                     0\n2024-03-03 00:00:00         156923           -0.0477  0.0812668                     0\n2024-03-02 00:00:00         164791           -0.1347  0.0812668                     0.0534\n2024-03-01 00:00:00         190443           -0.0687  0.0812668                     0\n2024-02-29 00:00:00         204489            0.2069  0.0812668                     0.1256\n2024-02-28 00:00:00         169434            0.0871  0.0812668                     0.0058\n2024-02-27 00:00:00         155865            0.0193  0.0812668                     0\n2024-02-26 00:00:00         152912           -0.0114  0.0812668                     0\n2024-02-25 00:00:00         154670            0.0128  0.0812668                     0\n2024-02-24 00:00:00         152718           -0.0221  0.0812668                     0"
    },
    {
      "role": "assistant",
      "content": "Revenue Summary:\nOn March 4, 2024, the daily revenue was $4444.65, a decrease of 26.36% from the previous day. This change is outside the standard deviation of 14.09% by 12.27%. This significant decrease in revenue could be due to a variety of factors, such as changes in user behavior, market conditions, or the performance of the recommended product feed. It's worth noting that this is the second consecutive day of revenue decline, following a 11.44% decrease on March 3, 2024.\n\nOrder Summary:\nThe total number of orders on March 4, 2024, was 154,549, a decrease of 1.51% from the previous day. This change is within the standard deviation of 8.13%. However, on March 2, 2024, there was a significant decrease in the number of orders by 13.47%, which was outside the standard deviation by 5.34%. This could be a contributing factor to the decrease in revenue observed on March 4, 2024.\n\nTrends:\nOver the past week, there have been fluctuations in both revenue and order count, with significant decreases observed on March 1 and 2, 2024. These fluctuations could be due to changes in the effectiveness of the recommended product feed, or external factors such as market conditions or user behavior. It would be beneficial to investigate these changes further to understand the underlying causes and to develop strategies to improve performance."
    }
  ],
  temperature=0.02,
  max_tokens=2730,
  top_p=0,
  frequency_penalty=0,
  presence_penalty=0
)